<a href="https://colab.research.google.com/github/mayorofdata/demos/blob/main/GH_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
# Import Libraries and data
import pandas as pd
import json
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
# Read and viewing the data
df = pd.read_json("/content/drive/My Drive/data/sample_data.json")
df

,inappproduct
0,"{'packageName': 'gamehive_game1', 'sku': 'game..."
1,"{'packageName': 'gamehive_game1', 'sku': 'game..."


In [68]:
# Extracting the required information from the data
records = [
           {
               "packageName": x["packageName"],
            "sku": x["sku"],
            "countryCode": k,
            "currency": v["currency"],
            "price": str(float(v["priceMicros"])/1000000)
    }
     for x in df["inappproduct"]
    for k, v in x["prices"].items()
    if isinstance(v, dict) and v.get("currency") is not None
]
app_df = pd.DataFrame(records)
app_df = app_df.reindex(columns=["packageName", "sku", "countryCode", "currency", "price"])
app_df

,packageName,sku,countryCode,currency,price
0,gamehive_game1,gamehive_game1_product1,DZ,DZD,6325.0
1,gamehive_game1,gamehive_game1_product1,AU,AUD,79.99
2,gamehive_game1,gamehive_game1_product1,CA,CAD,69.99
3,gamehive_game1,gamehive_game1_product1,IT,EUR,54.99
4,gamehive_game1,gamehive_game1_product1,JP,JPY,6000.0
5,gamehive_game1,gamehive_game1_product1,RU,RUB,3790.0
6,gamehive_game1,gamehive_game1_product1,SG,SGD,68.98
7,gamehive_game1,gamehive_game1_product1,KR,KRW,65000.0
8,gamehive_game1,gamehive_game1_product1,GB,GBP,48.99
9,gamehive_game1,gamehive_game1_product1,US,USD,49.99


In [138]:
df_ex = pd.read_json("/content/drive/My Drive/data/exchange_rates.json")
with open("/content/drive/My Drive/data/exchange_rates.json",'r') as f:
    data = json.loads(f.read())
# Flatten data
df_nested_list = pd.json_normalize(data, record_path =['exchange_rates_data'])
df2 = {'currency_code': 'USD', 'exchange_rate':1}
df_nested_list = df_nested_list.append(df2, ignore_index = True)
df3 = df_nested_list.append(df_nested_list, ignore_index= True)

In [140]:
result = pd.concat([app_df,df3], axis = 1)
result['price'] = result['price'].astype(float, errors = 'raise')
result['usd_price'] = result['price']*result['exchange_rate']
pd.set_option('display.float_format', '{:.2f}'.format)
result.drop(['currency_code', 'exchange_rate'], axis=1, inplace=True)
result

,packageName,sku,countryCode,currency,price,usd_price
0,gamehive_game1,gamehive_game1_product1,DZ,DZD,6325.00,754699.00
1,gamehive_game1,gamehive_game1_product1,AU,AUD,79.99,111.99
2,gamehive_game1,gamehive_game1_product1,CA,CAD,69.99,93.79
3,gamehive_game1,gamehive_game1_product1,IT,EUR,54.99,48.94
4,gamehive_game1,gamehive_game1_product1,JP,JPY,6000.00,671580.00
5,gamehive_game1,gamehive_game1_product1,RU,RUB,3790.00,241839.90
6,gamehive_game1,gamehive_game1_product1,SG,SGD,68.98,93.81
7,gamehive_game1,gamehive_game1_product1,KR,KRW,65000.00,74163700.00
8,gamehive_game1,gamehive_game1_product1,GB,GBP,48.99,37.72
9,gamehive_game1,gamehive_game1_product1,US,USD,49.99,49.99
